In [1]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

df_annotations = pd.read_csv("all_annotations.csv")


# Get data

In [2]:
# with open("protocols.pkl.gz", "rb") as f:
#     file_to_pages = pkl.load(f)

In [3]:
# import spacy
# nlp = spacy.blank("en")

In [4]:
# texts = []

# for i in range(len(df_annotations)):
#     file_name = df_annotations.file.iloc[i]
    
#     pages= file_to_pages[file_name]
    
#     text = ""
#     for page in pages:
#         doc = nlp(page)
        
#         is_include = [False] * len(doc)
#         for tok in doc:
#             if tok.text.lower() in {"phase"}:
#                 for token_index in range(tok.i - 20, tok.i + 20):
#                     if token_index >= 0 and token_index < len(doc):
#                         is_include[token_index] = True
        
#         for token_index in range(len(doc)):
#             if is_include[token_index]:
#                 text += doc[token_index].text + doc[token_index].whitespace_
                
                
#     if text == "":
#         print ("nothing found", file_name, df_annotations.phase.iloc[i])
#         text = " ".join(pages)
#         if len(text) > 100000:
#             text = text[:100000]
#     else:
#         print ("found", file_name, df_annotations.phase.iloc[i])

#     texts.append(text)
# df_annotations["text"] = texts

In [5]:
# df_annotations.to_csv("filtered_for_phase.csv.bz2")

In [6]:
df_annotations = pd.read_csv("filtered_for_phase.csv.bz2")

In [7]:
# del file_to_pages

In [8]:
df_annotations_phase_known = df_annotations[~df_annotations.phase.isna()]

In [9]:
len(df_annotations_phase_known), len(df_annotations)

(11591, 11924)

In [10]:
#df_annotations_num_arms_known = df_annotations_num_arms_known.sample(1000)

In [11]:
df_annotations_phase_known.phase.value_counts()

Phase 2            4317
Phase 3            2725
Phase 4            1461
Phase 1            1154
Phase 1/Phase 2     983
Not Applicable      519
Phase 2/Phase 3     297
Early Phase 1       135
Name: phase, dtype: int64

In [12]:
phase_map = {"Phase 2":"2",
"Phase 3":"3",
"Phase 4":"4",
"Phase 1":"1",
"Phase 1/Phase 2":"1.5",
"Not Applicable":"0",
"Phase 2/Phase 3":"2.5",
"Early Phase 1":"0.5"}
df_annotations_phase_known["phase_clean"] = df_annotations_phase_known["phase"].map(phase_map)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
len(phase_map)

8

In [14]:
df_train = df_annotations_phase_known[df_annotations_phase_known.train_val == "train"]

In [15]:
# TRAINING_DATA = [
#     ["My little kitty is so special", {"KAT0": True}],
#     ["Dude, Totally, Yeah, Video Games", {"KAT1": True}],
#     ["Should I pay $1,000 for the iPhone X?", {"KAT1": True}],
#     ["The iPhone 8 reviews are here", {"KAT1": True}],
#     ["Noa is a great cat name.", {"KAT0": True}],
#     ["We got a new kitten!", {"KAT0": True}]
# ]

TRAINING_DATA = []
for idx in range(len(df_train)):
    cats = {}
    for a in phase_map.values():
        cats[a] = 0
    cats[df_train.phase_clean.iloc[idx]] = 1
    text = df_train.text.iloc[idx]
    if len(text) > 1000000:
        text = text[:1000000]
    
    TRAINING_DATA.append([text , cats])

In [16]:
!pip install spacy

In [17]:
import spacy
# Add imports for example, as well as textcat config...
from spacy.training import Example
from spacy.pipeline.textcat import single_label_bow_config, single_label_default_config
from thinc.api import Config
import random

# labels should be one-hot encoded



# bow
# config = Config().from_str(single_label_bow_config)

# textensemble with attention
config = Config().from_str(single_label_default_config)

nlp = spacy.blank("en")
# now uses `add_pipe` instead
category = nlp.add_pipe("textcat", last=True, config=config)
for a in range(1, 6):
    category.add_label(str(a))


# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(100):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=4):
        texts = [nlp.make_doc(text) for text, entities in batch]
        annotations = [{"cats": entities} for text, entities in batch]

        # uses an example object rather than text/annotation tuple
        examples = [Example.from_dict(doc, annotation) for doc, annotation in zip(
            texts, annotations
        )]
        nlp.update(examples, losses=losses)
    if itn % 20 == 0:
        print(losses)



{'textcat': 184.25332070881677}


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-146c2dd21dfe>", line 43, in <module>
    nlp.update(examples, losses=losses)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/spacy/language.py", line 1170, in update
    proc.update(examples, sgd=None, losses=losses, **component_cfg[name])  # type: ignore
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/spacy/pipeline/textcat.py", line 245, in update
    bp_scores(d_scores)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/thinc/layers/chain.py", line 61, in backprop
    dX = callback(dY)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/thinc/layers/concatenate.py", line 72, in backprop
    gradient = bwd(dY)
 

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_val = df_annotations_phase_known[df_annotations_phase_known.train_val == "val"]

In [ ]:
predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
df_val["y_pred"] = [int(max(p, key=p.get)) for p in predictions]

In [ ]:
phase_ground_truths = df_val.phase_clean
pred_phase = df_val["y_pred"]

In [ ]:
acc = accuracy_score(phase_ground_truths, pred_phase)
print (f"Phase accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(phase_ground_truths, pred_phase)
plt.xticks(rotation=90)
;

In [ ]:
nlp.to_disk("phase_model_01_textcat.spacy")
